In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [22]:
deprem_df = pd.concat((pd.read_csv("data/2002-01-01_2013-01-01_1.5_9.0(1).txt", sep="\t"), 
                       pd.read_csv("data/2002-01-01_2013-01-01_1.5_9.0(2).txt", sep="\t")))
deprem_df.drop(columns=["No    "], inplace=True)

deprem_df["date"] = pd.to_datetime(deprem_df["Deprem Kodu"], format="%Y%m%d%H%M%S")
deprem_df["date"] = deprem_df["date"].dt.floor("min")
deprem_df=deprem_df.sort_values('date')

print(deprem_df.head(30))

          Deprem Kodu Olus tarihi  Olus zamani  Enlem  Boylam  Der(km)   xM  \
44074  20020101180600  2002.01.01  18:06:00.30  37.75   36.33     10.0  3.2   
44073  20020102032111  2002.01.02  03:21:11.90  39.40   27.96      7.0  2.7   
44072  20020102120234  2002.01.02  12:02:34.00  39.44   38.68     45.0  3.5   
44071  20020102143025  2002.01.02  14:30:25.80  37.06   35.94     29.0  2.3   
44070  20020103073705  2002.01.03  07:37:05.40  36.97   28.92     10.0  3.0   
44069  20020104002223  2002.01.04  00:22:23.90  37.11   36.49      2.0  3.0   
44068  20020104024807  2002.01.04  02:48:07.80  39.30   27.81      4.0  2.8   
44067  20020104040756  2002.01.04  04:07:56.10  39.98   26.72      6.0  3.0   
44066  20020104173626  2002.01.04  17:36:26.00  40.28   34.46      5.0  3.1   
44065  20020104204414  2002.01.04  20:44:14.10  40.84   30.89      7.0  3.4   
44064  20020105110411  2002.01.05  11:04:11.30  40.75   30.97     24.0  3.1   
44063  20020105133959  2002.01.05  13:39:59.60  38.4

In [39]:
deprem_gdf = gpd.GeoDataFrame(deprem_df, geometry=gpd.points_from_xy(deprem_df.Boylam, deprem_df.Enlem))
deprem_gdf = deprem_gdf.set_crs("EPSG:4326")
deprem_gdf["Energy"] = 2 ** (5.24 + 1.44 * deprem_gdf["xM"])

depth_year_mask = (deprem_gdf["Der(km)"] <= 20) & (deprem_gdf["date"].dt.year >= 2002 ) & ( deprem_gdf["date"].dt.year <= 2012 )
deprem_gdf = deprem_gdf[depth_year_mask].reset_index(drop=True)
deprem_gdf=deprem_gdf.set_index("date")


In [53]:
marmara_gdf = gpd.read_file("data\marmara_rect.geojson")
marmara_gdf = marmara_gdf.set_crs("EPSG:4326")

deprem_marmara_gdf = gpd.sjoin(deprem_gdf, marmara_gdf, predicate='within')

deprem_grouper = deprem_marmara_gdf.groupby(pd.Grouper(freq="d"))
temp_df=deprem_grouper.first()                     #just for getting  max xM and E_daily values
temp_df["E_daily"] = deprem_grouper["Energy"].max()
temp_df["xM"] = deprem_grouper["xM"].max()


start = np.datetime64('2002-01-01') 
end = np.datetime64('2013-01-01') # not included
timeSignal=np.arange(start, end, np.timedelta64(1, 'D'))
er_mag_df=pd.DataFrame(index=timeSignal).join(temp_df)


In [54]:
er_mag_df["E_daily"].to_csv("data\energy_release.csv")
er_mag_df["xM"].to_csv("data\magnitudes.csv")